In [1]:
import pandas as pd
import json
import numpy as np
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client.aoligei
movielens_collection = db.movielens_movies
reviewlens_collection = db.movielens_reviews

In [3]:
movies = pd.read_csv('./raw_data/movie_lens/movies.csv')

In [4]:
links = pd.read_csv('./raw_data/movie_lens/links.csv')

In [5]:
movies = movies.merge(links, on='movieId')

In [6]:
movies = movies.rename(columns={"imdbId": "_id", "movieId": "movieLens_Id"})

In [7]:
movies['_id'] = movies['_id'].astype(str)

In [8]:
movies['_id'] = 'tt' + movies._id.str[:]

In [9]:
ratings = pd.read_csv('./raw_data/movie_lens/ratings.csv')

In [10]:
ratings_permovie = ratings.groupby('movieId').rating.apply(list).reset_index()

In [11]:
movies = movies.merge(ratings_permovie,left_on='movieLens_Id', right_on='movieId')

In [12]:
movies = movies.merge(ratings.groupby('movieId').mean(),left_on='movieLens_Id', right_on='movieId')

In [13]:
movies = movies.drop(['movieId', 'userId','timestamp'], axis=1)

In [14]:
movies=movies.rename(columns={"rating_x": "individual_ratings", "rating_y": "avg_rating"})

In [15]:
movies

,movieLens_Id,title,genres,_id,tmdbId,individual_ratings,avg_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,tt114709,862.0,"[4.0, 4.0, 4.5, 2.5, 4.5, 3.5, 4.0, 3.5, 3.0, ...",3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,tt113497,8844.0,"[4.0, 4.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.5, 4.0, ...",3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,tt113228,15602.0,"[4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 3.0, ...",3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,tt114885,31357.0,"[3.0, 3.0, 3.0, 3.0, 1.0, 2.0, 1.5]",2.357143
4,5,Father of the Bride Part II (1995),Comedy,tt113041,11862.0,"[5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 2.0, 3.0, 4.0, ...",3.071429
...,...,...,...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,tt5476944,432131.0,[4.0],4.000000
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,tt5914996,445030.0,[3.5],3.500000
9721,193585,Flint (2017),Drama,tt6397426,479308.0,[3.5],3.500000
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,tt8391976,483455.0,[3.5],3.500000


In [16]:
ratings = ratings.merge(links[['movieId', 'imdbId']], on='movieId')

In [17]:
ratings = ratings.rename(columns={"movieId": "movieLens_Id"})

In [18]:
ratings_tojson = ratings.to_json(orient="records")
result_ratings = json.loads(ratings_tojson)

In [19]:
movies_tojson = movies.to_json(orient="records")
result = json.loads(movies_tojson)

In [20]:
movielens_collection.insert_many(result)
reviewlens_collection.insert_many(result_ratings)